# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [68]:
#Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [69]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [70]:
# Revisar la forma del dataset (número de filas y columnas)
print("Shape of dataset:", spaceship.shape)
# Tipos de datos de cada columna
print(spaceship.dtypes)
# Verificar valores nulos
print(spaceship.isnull().sum())
# Eliminar filas con valores faltantes
spaceship_cleaned = spaceship.dropna()

# Verificar que no haya más valores nulos
print(spaceship_cleaned.isnull().sum())

# Mostrar el nuevo tamaño del dataset
print("Shape of cleaned dataset:", spaceship_cleaned.shape)
# Extraer la letra del deck de la columna 'Cabin'
spaceship_cleaned['Deck'] = spaceship_cleaned['Cabin'].str[0]

# Verificar los valores únicos en la nueva columna 'Deck'
print(spaceship_cleaned['Deck'].unique())

# Revisar los valores únicos en la columna 'Deck'
print("Valores únicos en 'Deck':", spaceship_cleaned['Deck'].unique())

# Eliminar la columna original 'Cabin' si ya no es necesaria
spaceship_cleaned = spaceship_cleaned.drop(columns=['Cabin'])

# Revisar el DataFrame después de la limpieza
spaceship_cleaned.head()

# Eliminar las columnas 'PassengerId' y 'Name'
spaceship_cleaned = spaceship_cleaned.drop(columns=['PassengerId', 'Name'])

# Verificar el DataFrame después de eliminar las columnas
print(spaceship_cleaned.head())

# Crear dummies para columnas no numéricas
spaceship_encoded = pd.get_dummies(spaceship_cleaned, drop_first=True)

# Verificar el DataFrame después de aplicar get_dummies
spaceship_encoded.head()

Shape of dataset: (8693, 14)
PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object
PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64
PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64
Shape of cleaned dataset: (6606, 14)
['B' 'F' 'A' 'G' 'E' 'C' '

C:\Users\Usuario\AppData\Local\Temp\ipykernel_17968\1866529543.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spaceship_cleaned['Deck'] = spaceship_cleaned['Cabin'].str[0]


,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T
0,39.0,0.0,0.0,0.0,0.0,0.0,False,True,False,False,False,True,False,True,False,False,False,False,False,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,False,False,False,False,True,False,False,False,False,False,True,False,False
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,True,False,False,False,True,True,False,False,False,False,False,False,False
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,True,False,False,False,True,False,False,False,False,False,False,False,False
4,16.0,303.0,70.0,151.0,565.0,2.0,True,False,False,False,False,True,False,False,False,False,False,True,False,False


**Perform Train Test Split**

In [71]:
# Selección de características
X = spaceship.drop(columns=['PassengerId', 'Transported'])  # Excluir PassengerId y la variable objetivo
y = spaceship['Transported']  # Variable objetivo

# Realizar la división de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Mostrar la forma de los conjuntos resultantes
print("Forma del conjunto de entrenamiento:", X_train.shape)
print("Forma del conjunto de prueba:", X_test.shape)

Forma del conjunto de entrenamiento: (6954, 12)
Forma del conjunto de prueba: (1739, 12)


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Definir X y y
X = spaceship_encoded.drop(columns=['Transported'])  # Excluir la variable objetivo
y = spaceship_encoded['Transported']  # Variable objetivo

# Realizar la división de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Escalado de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Ajustar y transformar el conjunto de entrenamiento
X_test_scaled = scaler.transform(X_test)  # Solo transformar el conjunto de prueba

# 1. Bagging con un clasificador base (Random Forest)
bagging_model = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=50, random_state=42)
bagging_model.fit(X_train_scaled, y_train)

# Predicción y evaluación del modelo Bagging
y_pred_bagging = bagging_model.predict(X_test_scaled)
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
print(f"Precisión del modelo Bagging: {accuracy_bagging:.4f}")

# 2. Pasting con un clasificador base (Random Forest)
pasting_model = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=50, bootstrap=False, random_state=42)
pasting_model.fit(X_train_scaled, y_train)

# Predicción y evaluación del modelo Pasting
y_pred_pasting = pasting_model.predict(X_test_scaled)
accuracy_pasting = accuracy_score(y_test, y_pred_pasting)
print(f"Precisión del modelo Pasting: {accuracy_pasting:.4f}")


Precisión del modelo Bagging: 0.7859
Precisión del modelo Pasting: 0.7829


- Random Forests

In [73]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Crear el modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Ajustar el modelo al conjunto de entrenamiento
rf_model.fit(X_train_scaled, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred_rf = rf_model.predict(X_test_scaled)

# Evaluar el modelo
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Precisión del modelo Random Forest: {accuracy_rf:.4f}")

# Generar el reporte de clasificación
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_rf))

# Mostrar la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_rf)
print("\nMatriz de confusión:")
print(conf_matrix)


Precisión del modelo Random Forest: 0.7753

Reporte de clasificación:
              precision    recall  f1-score   support

       False       0.77      0.78      0.77       656
        True       0.78      0.77      0.78       666

    accuracy                           0.78      1322
   macro avg       0.78      0.78      0.78      1322
weighted avg       0.78      0.78      0.78      1322


Matriz de confusión:
[[511 145]
 [152 514]]


- Gradient Boosting

In [74]:
from sklearn.ensemble import GradientBoostingClassifier

# Crear el modelo de Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Ajustar el modelo al conjunto de entrenamiento
gb_model.fit(X_train_scaled, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred_gb = gb_model.predict(X_test_scaled)

# Evaluar el modelo
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Precisión del modelo Gradient Boosting: {accuracy_gb:.4f}")

# Generar el reporte de clasificación
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_gb))

# Mostrar la matriz de confusión
conf_matrix_gb = confusion_matrix(y_test, y_pred_gb)
print("\nMatriz de confusión:")
print(conf_matrix_gb)


Precisión del modelo Gradient Boosting: 0.7837

Reporte de clasificación:
              precision    recall  f1-score   support

       False       0.82      0.72      0.77       656
        True       0.75      0.85      0.80       666

    accuracy                           0.78      1322
   macro avg       0.79      0.78      0.78      1322
weighted avg       0.79      0.78      0.78      1322


Matriz de confusión:
[[471 185]
 [101 565]]


- Adaptive Boosting

In [75]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Crear el modelo de AdaBoost (sin especificar base_estimator)
ada_model = AdaBoostClassifier(n_estimators=100, random_state=42)

# Ajustar el modelo al conjunto de entrenamiento
ada_model.fit(X_train_scaled, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred_ada = ada_model.predict(X_test_scaled)

# Evaluar el modelo
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print(f"Precisión del modelo AdaBoost: {accuracy_ada:.4f}")

# Generar el reporte de clasificación
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_ada))

# Mostrar la matriz de confusión
conf_matrix_ada = confusion_matrix(y_test, y_pred_ada)
print("\nMatriz de confusión:")
conf_matrix_ada


c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Precisión del modelo AdaBoost: 0.7814

Reporte de clasificación:
              precision    recall  f1-score   support

       False       0.81      0.73      0.77       656
        True       0.76      0.83      0.79       666

    accuracy                           0.78      1322
   macro avg       0.78      0.78      0.78      1322
weighted avg       0.78      0.78      0.78      1322


Matriz de confusión:


array([[479, 177],
       [112, 554]], dtype=int64)

Which model is the best and why?

## Comparación de Modelos

- **Bagging** muestra la mejor precisión (0.7859) y un balance aceptable entre precisión y recall. Esto indica que tiene un buen rendimiento general.

- **Pasting** y **Gradient Boosting** tienen métricas similares, pero su precisión es inferior a la de Bagging. Sin embargo, Gradient Boosting tiene un recall más alto para la clase `False` (0.82), lo que indica que es mejor para identificar correctamente los negativos.

- **Random Forest** tiene la menor precisión (0.7753) y muestra un rendimiento inferior en comparación con los otros modelos.

- **AdaBoost** tiene métricas similares a Pasting y Gradient Boosting, pero un poco más bajo en precisión (0.7814).

## Conclusión

Bagging es el mejor modelo según las métricas presentadas, ya que tiene la mayor precisión general. Sin embargo, dependiendo del contexto del problema, también es importante considerar el recall y el F1-score. Por ejemplo, si el objetivo es minimizar los falsos negativos (predecir correctamente la clase `True`), entonces el **Gradient Boosting** podría ser más apropiado debido a su mejor recall para la clase `True`.
